# Stock trading agent

## Data
We have a folder from https://www.kaggle.com/datasets/camnugent/sandp500 which includes the prices every market open day from 08-02-2013 to 08-02-2018 fro over 500 companies. These are companies in the S&P 500 and are the top 500 markets in the US, furthermore they represent the shares of the most commonly traded companies in the stock market.

The data has 5 columns for each company in its own file named by its stock name:
- Date: yy-mm-dd
- Open: price of stock at market open
- High: Highest price of stock in day
- Low: Lowest price of stock in day
- Close: Price of closing
- Volume: How many stocks traded that day for that company
- Name: The ticker name of the company in the stock market

In [67]:
from pathlib import Path

def count_files(directory_path):
    path = Path(directory_path)
    count = len([p for p in path.iterdir() if p.is_file()])
    return count

directory = 'individual_stocks_5yr'
file_count = count_files(directory)
print(f"Number of files in '{directory}': {file_count}")

Number of files in 'individual_stocks_5yr': 505


### Reading the data
Here we read the csv files for the first 100 companies and store the csv file in a panda dataframe and then store the 100 dataframes in a simple list.

In [30]:
# Schema: date | open | high | low | close | volume | name

# import os

# folder_path = 'individual_stocks_5yr'

# entries = os.listdir(folder_path)

# import pandas as pd

# stock_prices = []

# for i in range(100):
#     filename = folder_path + "/" + entries[i]
#     try:
#         df = pd.read_csv(filename)
#         stock_prices.append(df)
#     except:
#         print("Error: Error reading file (" + filename + ")")

# # A vector with the days for conveinience later
# calendar = stock_prices[0]['date']

# # print(len(stock_prices))


In [31]:
# Schema: date | open | high | low | close | volume | name

from pathlib import Path
import pandas as pd

def read_data(folder_path, amount = None):
    path = Path(folder_path)
    files = []

    for f in path.iterdir():
        if f.is_file() and f.suffix.lower() == '.csv':
            filename = f
            try:
                df = pd.read_csv(filename)
                files.append(df)
                if  amount:
                    amount -= 1
                    if amount == 0:
                        return files
            except:
                print("Error: Error reading file (" + filename + ")")

    return files

In [32]:
def inspect_data(comp_list):
    def check_for_na(comp_df):
        total_nas = comp_df.isna().sum().sum()
        uniform = True

        if total_nas != 0:
            print(f"Null values found. File: {comp_df.iat[0, 6]}. Count: {total_nas}")
            uniform = False
        return uniform

    def check_dfs_na(comp_list):
        print("Checking for null values:")
        if comp_list:
            for df in comp_list:
                unifrom = check_for_na(df)
        else:
            print("Error: No dataframes in comp_list!")
    def check_length(comp_list):
        print("Checking sizes:")
        default_shape = comp_list[0].shape
        uniform = True

        for i, df in enumerate(comp_list):
            if df.shape != default_shape:
                print(f"Unequal size detected. Deffault: {default_shape}. Detected size: {df.shape} in file {df.iat[0, 6]}")
                uniform = False
        if uniform:
            print("All dfs same shape")            

    def check_dates(comp_list):
        print("Checking for unequal dates:")
        calendar = comp_list[0]['date']
        uniform = True

        for i, df in enumerate(comp_list):
            if not df['date'].equals(calendar):
                print(f"Unequal dates detected for file {df.iat[0, 6]}")
                uniform = False
        if uniform:
            print('All dates are equal')
    
    if not comp_list or len(comp_list) <= 1:
            return
    
    check_dfs_na(comp_list)
    check_length(comp_list)
    check_dates(comp_list)

The above code shows that there are indeed Null values and unequal data for some companies in our dataset.

In [ ]:
def clean_data(comp_list):
    if not comp_list or len(comp_list) == 0:
        print("Error: Input empty")
        return
    
    print("Cleaning dataframes from NaNs")
    counter = 0
    clean_list = []

    for df in comp_list:
        if df.isnull().values.any():
            print(f"NaNs detected in {df.iat[0, 6]}, will be dropped")
            counter += 1
        else:
            clean_list.append(df)

    print(f"NaN dataframes dropped {counter}, will equalize length now.")

    min_days = 730
    valid_companies = [df for df in clean_list if df.shape[0] >= min_days]

    date_sets = [set(df['date']) for df in valid_companies]

    common_dates = set.intersection(*date_sets)

    aligned_dfs = [
        df[df['date'].isin(common_dates)].sort_values('date').reset_index(drop=True)
        for df in valid_companies
    ]

    return aligned_dfs


In [68]:
comp_list = (read_data(folder_path='individual_stocks_5yr', amount=505))
inspect_data(comp_list)

Checking for null values:
Null values found. File: WRK. Count: 3
Null values found. File: FTV. Count: 3
Null values found. File: ES. Count: 1
Null values found. File: UA. Count: 3
Null values found. File: DHR. Count: 4
Null values found. File: O. Count: 4
Null values found. File: VRTX. Count: 3
Null values found. File: BHF. Count: 3
Null values found. File: REGN. Count: 3
Checking sizes:
Unequal size detected. Deffault: (1259, 7). Detected size: (975, 7) in file GOOG
Unequal size detected. Deffault: (1259, 7). Detected size: (781, 7) in file QRVO
Unequal size detected. Deffault: (1259, 7). Detected size: (1063, 7) in file ALLE
Unequal size detected. Deffault: (1259, 7). Detected size: (1257, 7) in file ORCL
Unequal size detected. Deffault: (1259, 7). Detected size: (917, 7) in file INFO
Unequal size detected. Deffault: (1259, 7). Detected size: (1257, 7) in file BMY
Unequal size detected. Deffault: (1259, 7). Detected size: (1197, 7) in file IQV
Unequal size detected. Deffault: (1259, 

In [69]:
clean_list = clean_data(comp_list)
inspect_data(clean_list)
comp_count = len(clean_list)
print(f"We now have: {len(clean_list)}")

Cleaning dataframes from NaNs
NaNs detected in WRK, will be dropped
NaNs detected in FTV, will be dropped
NaNs detected in ES, will be dropped
NaNs detected in UA, will be dropped
NaNs detected in DHR, will be dropped
NaNs detected in O, will be dropped
NaNs detected in VRTX, will be dropped
NaNs detected in BHF, will be dropped
NaNs detected in REGN, will be dropped
NaN dataframes dropped 9, will equalize length now.
Checking for null values:
Checking sizes:
All dfs same shape
Checking for unequal dates:
All dates are equal
We now have: 484


Now all our data for all chosen companies are equal.

In [ ]:
def simulate_market_at_day(num_days, trading_budget, risk, day, n, threshold):
    # A dict to store all purchases, using stock name as key, and list of lists as values.
    # Where each list would store [amout of shares bought, purchase price, date of purchase]
    purchases = {}
    purchase_history = {}
    sells = {}
    budget = trading_budget
    max_purchase = 100
    fee_rate = 0.001

    def calc_momentum(comp_df, curr, n):
        price_n_ago = comp_df.iat[curr - n, 1]

        curr_price = comp_df.iat[curr, 1]
                
        return (curr_price - price_n_ago) / price_n_ago

    def buy_amount(comp_df, curr, momentum):
        curr_price = comp_df.iat[curr, 1]
        date = comp_df.iat[curr, 0]

        momentum_risk = risk * (1 + min(momentum, 0.05) / 0.05)

        amount_of_shares = (momentum_risk * budget) / curr_price
        if amount_of_shares * curr_price > max_purchase:
            amount_of_shares = max_purchase / curr_price
        return  [amount_of_shares, curr_price, date]

    def make_purchase(comp_df, curr, momentum):
        stock_name = comp_df.iat[0, 6]
        purchase_record = buy_amount(comp_df, curr, momentum)
        nonlocal budget

        if stock_name in purchases and purchases[stock_name]:
            purchases[stock_name].append(purchase_record)
        else:
            purchases[stock_name] = [purchase_record, ]
        budget -= (purchase_record[0] * purchase_record[1]) * (1 + fee_rate)

    def make_sell_record(stock_name, purchase_records, total_amount):
        if stock_name in purchase_history:
            for record in purchase_records:
                purchase_history[stock_name].append(record)
        else:
                purchase_history[stock_name] = purchase_records
        if stock_name in sells:
            sells[stock_name].append(total_amount)
        else:
            sells[stock_name] = [total_amount, ]

    def sell_shares(stock_name, curr_price):
        nonlocal budget
        purchase_records = purchases[stock_name]
        total_amount = 0
        for record in purchase_records:
            total_amount += curr_price * record[0]
        budget += total_amount * (1 - fee_rate)
        make_sell_record(stock_name, purchase_records, total_amount)
        del purchases[stock_name]
        
    def make_sell(comp_df, curr):
        stock_name = comp_df.iat[0, 6]
        curr_price = comp_df.iat[curr, 1]

        if stock_name in purchases and purchases[stock_name]:
            sell_shares(stock_name, curr_price)

    def movement_decision(comp_df, curr, n):
        momentum = calc_momentum(comp_df=comp_df, curr=curr, n=n)
        if momentum > threshold:
            make_purchase(comp_df, curr, momentum)
        elif momentum < -threshold:
            make_sell(comp_df, curr)
    
    def sell_all_shares(last_day):
        for purchase in purchases:
            
    
    for i in range(day, num_days, 1):
        if i < n:
            continue
        for j in range(len(clean_list)):
            movement_decision(clean_list[j], i, n)

    sell_all_shares(num_days - 1)

    initial_budget = trading_budget

    last_day_index = num_days - 1

    unrealized = 0
    for stock_name, buys in purchases.items():

        comp_df = next(df for df in clean_list if df.iat[0, 6] == stock_name)
        curr_price = comp_df.iat[last_day_index, 4]
        
        for qty, _, _ in buys:
            unrealized += qty * curr_price

    portfolio_value = budget + unrealized
    profit = portfolio_value - initial_budget

    count = 0
    total_sell_operations = [count + len(s) for s in sells]
    count = 0
    total_purchase_operations = [count + len(s) for s in purchase_history]

    print("Final cash:", budget)
    print("Unrealized value:", unrealized)
    print("Total portfolio value:", portfolio_value)
    print("Total profit:", profit)

    return portfolio_value


In [ ]:
    
simulate_market_at_day(734, 5000, 0.05, 0, 3, 0.01)
simulate_market_at_day(734, 5000, 0.05, 0, 5, 0.01)
simulate_market_at_day(734, 5000, 0.05, 0, 10, 0.01)
simulate_market_at_day(734, 5000, 0.05, 0, 20, 0.01)

In [121]:
simulate_market_at_day(734, 5000, 0.05, 0, 3, 0.2)
simulate_market_at_day(734, 5000, 0.05, 0, 5, 0.2)
simulate_market_at_day(734, 5000, 0.05, 0, 10, 0.2)
simulate_market_at_day(734, 5000, 0.05, 0, 20, 0.2)

Final cash: 148.69765850581183
Unrealized value: 5973.3644415350145
Total portfolio value: 6122.062100040826
Total profit: 1122.0621000408264
Final cash: 3.7885352507813135
Unrealized value: 6435.225296178847
Total portfolio value: 6439.013831429628
Total profit: 1439.0138314296282
Final cash: 0.8376526180457003
Unrealized value: 8157.846258382721
Total portfolio value: 8158.683911000767
Total profit: 3158.683911000767
Final cash: 8.324599077722729
Unrealized value: 7608.306465662932
Total portfolio value: 7616.631064740655
Total profit: 2616.6310647406553


7616.631064740655

In [120]:
simulate_market_at_day(364, 5000, 0.05, 0, 3, 0.01)
simulate_market_at_day(364, 5000, 0.05, 0, 5, 0.01)
simulate_market_at_day(364, 5000, 0.05, 0, 10, 0.01)
simulate_market_at_day(364, 5000, 0.05, 0, 20, 0.01)

Final cash: 379.7536956051303
Unrealized value: 4031.422870349891
Total portfolio value: 4411.176565955021
Total profit: -588.823434044979
Final cash: 127.9721404641229
Unrealized value: 4192.916291565801
Total portfolio value: 4320.888432029924
Total profit: -679.1115679700761
Final cash: 5.465101964825226
Unrealized value: 4489.238130947475
Total portfolio value: 4494.7032329123
Total profit: -505.29676708769966
Final cash: 0.015999246569712078
Unrealized value: 4722.683958210347
Total portfolio value: 4722.699957456916
Total profit: -277.3000425430837


4722.699957456916

In [119]:
simulate_market_at_day(364, 5000, 0.3, 0, 3, 0.20)
simulate_market_at_day(364, 5000, 0.3, 0, 5, 0.20)
simulate_market_at_day(364, 5000, 0.3, 0, 10, 0.20)
simulate_market_at_day(364, 5000, 0.3, 0, 20, 0.20)

Final cash: 10.305291583571059
Unrealized value: 5214.561813224154
Total portfolio value: 5224.867104807726
Total profit: 224.8671048077258
Final cash: 23.93035771043592
Unrealized value: 5007.86400327737
Total portfolio value: 5031.794360987806
Total profit: 31.794360987805703
Final cash: 1.4055663114280845e-10
Unrealized value: 5606.394207078959
Total portfolio value: 5606.3942070791
Total profit: 606.3942070790999
Final cash: 5.1036063552414395e-30
Unrealized value: 6228.694225302273
Total portfolio value: 6228.694225302273
Total profit: 1228.6942253022726


6228.694225302273

In [ ]:
# # num_days = clean_list[0].shape[0] - 1
# num_days = 700


# n = 5

# budget = 5000
# purchase_amount = 100

# purchases = {}
# sells = []

# momentum_day = []
# for i in range(num_days):
#     momentums = {}

#     for j in range(comp_count):
#         if i >= n:
#             comp_df = clean_list[j]

#             price_n_ago = comp_df.iat[i - n, 1]

#             curr_price = comp_df.iat[i, 1]

#             curr_date = comp_df.iat[i, 0]
            
#             momentum = (curr_price - price_n_ago) / price_n_ago
#             stock_name = comp_df.iat[i, 6]
#             momentums[stock_name] = momentum
#             if momentum > 0 and budget > purchase_amount:
#                 # Key is stock name, list contains [how many stocks purchased, price at purchase, purchase date]
#                 new_purchase = [purchase_amount / curr_price, curr_price, curr_date]
#                 if stock_name in purchases:
#                     purchases[stock_name].append(new_purchase)
#                 else:
#                     purchases[stock_name] = [new_purchase, ]
#                 budget -= purchase_amount

#             elif momentum < 0 and stock_name in purchases and purchases[stock_name]:
#                 old_purchases = [purchase for purchase in purchases[stock_name]]

#                 purchase_profit = 0
#                 for old_purchase in old_purchases:
#                     purchase_profit += old_purchase[0] * curr_price
#                 new_sell = {stock_name: [purchase_profit / curr_price, purchase_profit, curr_date]}
#                 sells.append(new_sell)
#                 del purchases[stock_name]
#                 budget += purchase_profit
#     momentum_day.append(momentums)

# # print(budget)

# initial_budget = 5000

# last_day_index = num_days - 1

# unrealized = 0
# for stock_name, buys in purchases.items():

#     comp_df = next(df for df in clean_list if df.iat[0, 6] == stock_name)
#     curr_price = comp_df.iat[last_day_index, 4]
    
#     for qty, _, _ in buys:
#         unrealized += qty * curr_price

# portfolio_value = budget + unrealized
# profit = portfolio_value - initial_budget

# print("Final cash:", budget)
# print("Unrealized value:", unrealized)
# print("Total portfolio value:", portfolio_value)
# print("Total profit:", profit)



Final cash: 69.9612822963663
Unrealized value: 5737.677901247763
Total portfolio value: 5807.639183544129
Total profit: 807.6391835441291


In [ ]:
# Baseline method
def baseline_invest(trading_budget=5000, n = 100):
    if not clean_list:
        print("Error: clean_list is empty.")
        return

    per_company_budget = trading_budget / len(clean_list)

    total_portfolio_value = 0

    for comp in clean_list:
        start_price = comp.iat[0, 4]
        end_price = comp.iat[n - 1, 4]

        shares = per_company_budget / start_price

        position_value = shares * end_price * (1 - 0.01)
        total_portfolio_value += position_value

    profit = total_portfolio_value - trading_budget

    print(f"Baseline final portfolio value: {total_portfolio_value}")
    print(f"Baseline profit: {profit}")

    return total_portfolio_value

In [109]:
baseline_invest(5000, 734)

Baseline final portfolio value: 6523.929488196036
Baseline profit: 1523.929488196036


6523.929488196036